In [ ]:
import cv2
import os
from matplotlib.rcsetup import validate_markevery
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
img = []
label = []
files = os.listdir("alphabets")
for i in tqdm(range(len(files))):
      files_alpha = os.listdir("alphabets"+"/"+files[i])
      for j in  tqdm(range(len(files_alpha))):
           sub_files_alpha = "alphabets"+"/"+files[i]+"/"+files_alpha[j] 
           image = cv2.imread(sub_files_alpha)
           image = cv2.resize(image,(96,96))
           image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
           img.append(image)
           label.append(i)    #loading dataset


In [ ]:
img = np.array(img)
label = np.array(label,dtype = "int")   

In [ ]:
x_train,x_test,y_train,y_test =  train_test_split(img,label,test_size=0.20,shuffle=True)
del img
del label
import gc
gc.collect()     #split the dataset into train and test

In [ ]:
from tensorflow import keras
from keras.applications import EfficientNetB0 
from keras.models import Sequential,Model,load_model
from keras import layers,optimizers,applications,utils
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint 
def get_Model():
        k = EfficientNetB0(include_top = False,input_shape = (96,96,3))
        model = Sequential()
        model.add(k)          
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dropout(0.3))
        model.add(Dense(25,activation = "sigmoid"))
        model.summary()
        return model         #create the model

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, 25,dtype = "int")  #encoding for softmax
y_test = to_categorical(y_test, 25,dtype = "int")

In [ ]:
model = get_Model()
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])  #compile the model

In [ ]:
path = "trained_model20/model"    #create check point
mcheck = ModelCheckpoint(filepath = path,monitor = "val_accuracy", mode = "auto",save_best_only =True,save_weights_only = True)

In [ ]:
mreduceLR = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.9,
    patience=3,
    verbose=1,
    mode="auto",           #reduce learning rate if the metric stopped improving
    cooldown=0,
    min_lr=0.00001,
)

In [ ]:
model_fit = model.fit(x_train,y_train,validation_data = (x_test,y_test),batch_size = 32,epochs =5,callbacks = [mcheck,mreduceLR])         

In [ ]:
model.save('new_model20.h5')

In [ ]:
from tensorflow import keras
from keras.applications import EfficientNetB0 
from keras.models import Sequential,Model,load_model
from keras import layers,optimizers,applications,utils
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint 
new_model = keras.models.load_model('new_model20.h5')

In [ ]:
import cv2
import numpy as np
import math
import time
import numpy
from cvzone.HandTrackingModule import HandDetector
imgwhite = 'A.jpg.jpg'
list = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
image = cv2.imread(imgwhite)
image = cv2.resize(image,(96,96))
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image = image.reshape(-1,96,96,3)
preds = new_model.predict(image)
j = numpy.argmax(preds)
print(list[j])  

In [ ]:
import cv2
import numpy as np
import math
import time
import numpy
import ctypes
from cvzone.HandTrackingModule import HandDetector
cap  = cv2.VideoCapture(0)
detect = HandDetector(maxHands = 1)
ofs = 30
imgsize = 500
count = 0
list = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
user32 = ctypes.windll.user32
screen_width, screen_height = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
kar=True
k = cv2.waitKey(1)
s = ''
while True:
    time.sleep(0.1)
    suc,img = cap.read()
    frame_height, frame_width, _ = np.shape(img)
    scaleWidth = float(screen_width)/float(frame_width)
    scaleHeight = float(screen_height)/float(frame_height)

    if scaleHeight>scaleWidth:
       imgScale = scaleWidth

    else:
       imgScale = scaleHeight

    newX,newY = frame_width*imgScale, frame_height*imgScale
    img = cv2.resize(img,(int(newX),int(newY)))
    
    cv2.namedWindow("image", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("image",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
    
    hand= detect.findHands(img,draw = False)
    if hand:  
       p = hand[0]
       x,y,w,h = p["bbox"]
       imgwhite = np.ones((imgsize,imgsize,3),np.uint8)*255
       imgcrop = img[y-ofs:y+h+ofs,x-ofs:x+w+ofs]
       ratio = h/w
      
       if ratio>1:
        k = imgsize/h
        w = math.ceil(k*w)
        imgresize = cv2.resize(imgcrop,(w,imgsize))
        imgresizeshape = imgresize.shape
        imgwhite[0:imgresizeshape[0], math.ceil((imgsize-imgresizeshape[1])/2):math.ceil((imgsize+imgresizeshape[1])/2) ] = imgresize

        image = cv2.resize(imgwhite,(96,96))
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = image.reshape(-1,96,96,3)
        preds = new_model.predict(image)
        max = preds[0][0]
        j = numpy.argmax(preds[0])
        
       else:
        k = imgsize/w
        h = math.ceil(k*h)
        imgresize = cv2.resize(imgcrop,(imgsize,h))
        imgresizeshape = imgresize.shape
        imgwhite[ math.ceil((imgsize-h)/2):math.ceil((imgsize+h)/2),0:imgsize ] = imgresize
        image = cv2.resize(imgwhite,(96,96))
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = image.reshape(-1,96,96,3)
        preds = new_model.predict(image)
        j = numpy.argmax(preds[0])
    image = cv2.putText(img,list[j]+"       "+s,(90,90),cv2.FONT_HERSHEY_DUPLEX,3,(58,179,255),3)
    cv2.imshow("image",image)
    k = cv2.waitKey(1)
    if k & 0xFF == ord('q'):     
       s = s+'{}'.format(list[j])
    if k & 0xFF ==ord('s'):
       s = s[:-1]  
    if k & 0xFF == ord('z'):
       break
      
cap.release()
print(s)      
cv2.destroyAllWindows() 

    

In [ ]:
pip install mediapipe